<a href="https://colab.research.google.com/github/mille055/AIPI540_individual_project/blob/main/notebooks/AIPI540_IP_fusion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!git clone 'https://github.com/mille055/AIPI540_individual_project.git'

Cloning into 'AIPI540_individual_project'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 343 (delta 24), reused 34 (delta 12), pack-reused 291
Receiving objects: 100% (343/343), 91.91 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [2]:
!pip install pydicom==2.1.2
!pip install monai seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
import monai
import pickle
import glob
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import os
import copy

import monai
from monai.data import DataLoader, ImageDataset
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType


import warnings
warnings.filterwarnings("ignore")

In [13]:
%cd ..
%ls

/Users/cmm/Documents/GitHub/AIPI540_individual_project
README.md         data/             notebooks/        scripts/
assets/           models/           requirements.txt


In [30]:
#local imports
import sys
sys.path.append('/Users/cmm/Documents/GitHub/AIPI540_individual_project/scripts/')

from config import *
from utils import *

# from AIPI540_individual_project.scripts.train_pixel_model import train_model
# from AIPI540_individual_project.scripts.train_text_model import load_text_data, train_text_model, list_incorrect_text_predictions
# from AIPI540_individual_project.scripts.utils import *

# Load Datasets


In [22]:
# for colab
# train_datafile = '/content/AIPI540_individual_project/data/X_train02282023.pkl'
# test_datafile = '/content/AIPI540_individual_project/data/X_test02282023.pkl'

In [39]:
# for local
train_datafile = './data/trainfiles.csv'
test_datafile = './data/testfiles.csv'

In [ ]:
def load_csv_dataset(train_file, test_file, val_lists = None):
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    train_df.drop('Unnamed: 0', axis=1, inplace=True)
    test_df.drop('Unnamed: 0', axis=1, inplace=True)

    if val_lists
        val_df = train_df[train_df.patientID.isin(val_lists)]
        train_df = train_df[~train_df.index.isin(val_df.index)] 
    else:
        train_set, val_set = next(GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 42).split(train_df, groups=train_df['patientID']))
        train_df, val_df = train_set, val_set
    

In [44]:
train, test = load_csv_dataset(train_datafile, test_datafile)

In [42]:
train.columns.tolist()

['fname_x',
 'StudyInstanceUID',
 'SeriesInstanceUID',
 'SeriesNumber',
 'SeriesDescription_x',
 'AcquisitionNumber',
 'InstanceNumber',
 'ImageOrientationPatient',
 'MRAcquisitionType',
 'AngioFlag',
 'SliceThickness',
 'RepetitionTime',
 'EchoTime',
 'EchoTrainLength',
 'PixelSpacing',
 'ContrastBolusAgent',
 'InversionTime',
 'seq_E',
 'seq_EP',
 'seq_G',
 'seq_IR',
 'seq_P',
 'seq_R',
 'seq_S',
 'seq_SE',
 'var_E',
 'var_K',
 'var_N',
 'var_O',
 'var_OSP',
 'var_P',
 'var_S',
 'var_SK',
 'var_SP',
 'var_SS',
 'opt_2',
 'opt_A',
 'opt_ACC_GEMS',
 'opt_D',
 'opt_EDR_GEMS',
 'opt_EPI_GEMS',
 'opt_F',
 'opt_FAST_GEMS',
 'opt_FC',
 'opt_FC_SLICE_AX_GEMS',
 'opt_FILTERED_GEMS',
 'opt_FS',
 'opt_FT_GEMS',
 'opt_I',
 'opt_IDEAL_GEMS',
 'opt_MP_GEMS',
 'opt_MRCP_GEMS',
 'opt_NPW',
 'opt_P',
 'opt_PFF',
 'opt_PFP',
 'opt_RTR_GEMS',
 'opt_S',
 'opt_SAT2',
 'opt_SAT_GEMS',
 'opt_SEQ_GEMS',
 'opt_SFS',
 'opt_SP',
 'opt_SS_GEMS',
 'opt_T',
 'opt_TRF_GEMS',
 'opt_VASCTOF_GEMS',
 'opt_W',
 'opt_X'

In [43]:
train_label_df = train[['fname_x', 'filename', 'Parent_folder', 'patientID', 'exam', 'series', 'SeriesDescription_y', 'plane', 'seq_label', 'contrast', 'original_label_code', 'ap_label_code']]
train_label_df.rename(columns={'SeriesDescription_y':'SeriesDescription'}, inplace=True)
train_label_df

,fname_x,filename,Parent_folder,patientID,exam,series,SeriesDescription,plane,seq_label,contrast,original_label_code,ap_label_code
0,Abdominal MRI data/104/exam1/18 (35-68)/0038.dcm,0038.dcm,Abdominal MRI data,104,exam1,18(35-68),ax diff_TRACEW,ax,dwi,0,8,8
1,Abdominal MRI data/104/exam1/18 (1-34)/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,18(1-34),ax diff_TRACEW,ax,dwi,0,8,8
2,Abdominal MRI data/104/exam1/20/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,20,ax equilibrium,ax,equilibrium,1,16,16
3,Abdominal MRI data/104/exam1/9/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,9,ax haste bh,ax,t2,0,9,9
4,Abdominal MRI data/104/exam1/11/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,11,ax t2 triggerred,ax,t2,0,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...
1729,Abdominal MRI data/094/exam1/3/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,3,cor haste,cor,t2,0,7,7
1730,Abdominal MRI data/094/exam1/12/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,12,ax dyn,ax,early dynamic,1,2,2
1731,Abdominal MRI data/094/exam1/13/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,13,ax dyn,ax,early dynamic,1,2,2
1732,Abdominal MRI data/094/exam1/5/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,5,ax dixon_opp,ax,opposed phase,0,19,19


In [45]:
test_label_df = test[['fname_x', 'filename', 'Parent_folder', 'patientID', 'exam', 'series', 'SeriesDescription_y', 'plane', 'seq_label', 'contrast', 'original_label_code', 'ap_label_code']]
test_label_df.rename(columns={'SeriesDescription_y':'SeriesDescription'}, inplace=True)
test_label_df

,fname_x,filename,Parent_folder,patientID,exam,series,SeriesDescription,plane,seq_label,contrast,original_label_code,ap_label_code
0,Abdominal MRI data/102/exam1/16 (1-40)/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,16(1-40),ax diff_TRACEW,ax,dwi,0,8,8
1,Abdominal MRI data/102/exam1/16 (41-80)/0073.dcm,0073.dcm,Abdominal MRI data,102,exam1,16(41-80),ax diff_TRACEW,ax,dwi,0,8,8
2,Abdominal MRI data/102/exam1/18/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,18,ax equilibrium new,ax,equilibrium,1,16,16
3,Abdominal MRI data/102/exam1/11/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,11,ax dyn new,ax,early dynamic,1,3,2
4,Abdominal MRI data/102/exam1/7/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,7,ax dixon_W,ax,unknown,0,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...
427,Abdominal MRI data/058/exam1/16 (1-36)/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,16(1-36),ax diff_TRACEW,ax,dwi,0,8,8
428,Abdominal MRI data/058/exam1/2/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,2,cor haste,cor,t2,0,7,7
429,Abdominal MRI data/058/exam1/13/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,13,ax dyn new,ax,early dynamic,1,2,2
430,Abdominal MRI data/058/exam1/5/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,5,ax dixon_in,ax,in phase,0,14,14


In [49]:
# for colab
# trainids = pd.read_csv('/content/AIPI540_individual_project/data/trainfiles.csv')
# for local
trainids = pd.read_csv('./data/trainfiles.csv')


In [50]:
trainids.iloc[6].file_info

'/volumes/cm7/Abdominal_MRI_dataset_split/train/104/exam1/18 (35-68)/0059.dcm'

In [51]:
val_df = pd.read_pickle('/content/AIPI540_individual_project/data/val_dicom_data.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/content/AIPI540_individual_project/data/val_dicom_data.pkl'

In [38]:
val_df

,label_code_x,predictions,patientID,series,img_label_code,GT label,prediction label,probability,original_label_code,SeriesDescription_y,plane,seq_label,contrast,label_code_y,final_prediction
0,8,8,41,16(37-72),8,dwi,dwi,0.976494,8,ax diff_TRACEW,ax,dwi,0,8,8
1,30,30,41,18,16,T1 post con,T1 post con fat sat,0.956994,16,ax equilibrium new,ax,equilibrium,1,16,16
2,30,30,41,11,3,T1 post con,T1 post con fat sat,0.956994,3,ax dyn new,ax,early dynamic,1,3,3
3,25,25,41,7,25,T1 fat sat,T1 fat sat,0.957891,25,ax dixon_W,ax,unknown,0,25,25
4,11,11,41,6,11,dixon fat,dixon fat,0.955352,11,ax dixon_F,ax,unknown,0,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,9,9,66,12,9,t2,t2,0.977097,9,ax t2 resp trig,ax,t2,0,9,9
350,7,7,66,2,17,t2,t2,0.914605,7,cor haste,cor,t2,0,7,7
351,11,11,66,5,11,dixon fat,dixon fat,0.834404,11,ax dixon top_F_FIL,ax,unknown,0,11,11
352,0,0,66,14,0,adc,adc,0.449541,0,ax diff_ADC,ax,dwi,0,0,0


In [ ]:
### does not have the filename colums.... will look for another
